In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_page_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup
url = "https://www.meff.es/esp/Derivados-Financieros/Ficha/FIEM_MiniIbex_35"
main_soup = fetch_page_content(url)





In [2]:
def extract_expiration_dates(soup):
    call_expiration_dates = {}
    put_expiration_dates = {}
    select_elem = soup.find('select', {'id': 'OpStrike'})

    for option in select_elem.find_all('option'):
        value = option['value']
        date = option.text

        if value.startswith('OCE'):
            call_expiration_dates[value] = date
        elif value.startswith('OPE'):
            put_expiration_dates[value] = date

    return call_expiration_dates, put_expiration_dates

call_expiration_dates, put_expiration_dates = extract_expiration_dates(main_soup)


In [33]:
put_expiration_dates

{'OPE20230421': '21/04/2023',
 'OPE20230428': '28/04/2023',
 'OPE20230505': '05/05/2023',
 'OPE20230512': '12/05/2023',
 'OPE20230519': '19/05/2023',
 'OPE20230616': '16/06/2023',
 'OPE20230915': '15/09/2023',
 'OPE20231215': '15/12/2023',
 'OPE20240315': '15/03/2024',
 'OPE20240621': '21/06/2024'}

In [34]:
def scrape_options_data(soup, expiration_value):
    tables = soup.find_all('table', {'class': 'Precios'})
    options_data = []
    print(tables)
    for table in tables:
        table_id = table.get('id')
        if table_id == 'tblOpciones':
            headers = [th.text for th in table.find_all('th')]
            for row in table.find('tbody').find_all('tr'):
                data = [td.text.strip() for td in row.find_all('td')]
                options_data.append(data)

    df = pd.DataFrame(options_data, columns=headers[:len(options_data[0])])
    df['expiration_date'] = expiration_value
    
    if len(headers) > len(df.columns):
        missing_columns = headers[len(df.columns):]
        for col in missing_columns:
            df[col] = None

    return df
date =  "21/04/2023"
scrape_options_data(main_soup, date)


[<table cellpadding="1" cellspacing="1" class="Precios" id="Contenido_Contenido_tblFuturos">
<caption>
			FUTUROS <small class="fecha">18/04/2023 19:44:18</small>
</caption><thead>
<tr>
<th rowspan="2" scope="col">Vencimiento</th><th rowspan="2" scope="col">Tipo</th><th colspan="3">Compra</th><th colspan="3">Venta</th><th rowspan="2" scope="col">Últ.</th><th rowspan="2" scope="col">Vol.</th><th rowspan="2" scope="col">Aper.</th><th rowspan="2" scope="col">Máx.</th><th rowspan="2" scope="col">Min.</th><th rowspan="2" scope="col">Ant.</th>
</tr><tr>
<th scope="col">Ord.</th><th scope="col">Vol.</th><th scope="col">Precio</th><th scope="col">Precio</th><th scope="col">Vol.</th><th scope="col">Ord.</th>
</tr>
</thead><tbody>
<tr class="text-right">
<td class="text-center colVcto" nowrap="nowrap">21 abr. 2023</td><td class="text-center">Difer.</td><td class="colCompra">4</td><td class="colCompra">13</td><td class="colCompra">9.435,00</td><td class="colVenta">9.440,00</td><td class="colVenta

,Strike,Compra,Venta,Últ.,Vol.,Aper.,Máx.,Min.,Ant.,Ord.,Vol.,Precio,Precio,expiration_date
0,"7.900,00",-,-,-,"9,00",2,1,-,0,None,-,-,-,21/04/2023
1,"8.300,00",-,-,-,-,-,-,"1.109,00",1,None,"1.109,00","1.109,00","1.076,00",21/04/2023
2,"8.500,00",-,-,-,"2,00",1,1,-,0,None,-,-,-,21/04/2023
3,"8.500,00",-,-,-,-,-,-,"904,00",1,None,"904,00","904,00","877,00",21/04/2023
4,"8.600,00",-,-,-,"3,00",1,1,-,0,None,-,-,-,21/04/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629,"9.400,00",-,-,-,-,-,-,"891,00",1,None,"891,00","891,00","842,00",21/04/2023
630,"9.400,00",-,-,-,-,-,-,"933,00",2,None,"933,00","929,00","901,00",21/04/2023
631,"9.400,00",-,-,-,-,-,-,"990,00",1,None,"990,00","990,00","950,00",21/04/2023
632,Volumen Total,,,None,None,None,None,None,None,None,None,None,None,21/04/2023


In [16]:
def extract_data_from_rows(soup):
    rows = soup.find_all('tr', {'class': 'text-right'})
    put_data = {}
    call_data = {}

    for row in rows:
        if 'data-tipo' in row.attrs:
            data_tipo = row['data-tipo']
            td_values = [td.text.strip() for td in row.find_all('td')]

            if data_tipo.startswith('OPE'):
                if data_tipo in put_data:
                    put_data[data_tipo].append(td_values)
                else:
                    put_data[data_tipo] = [td_values]
            elif data_tipo.startswith('OCE'):
                if data_tipo in call_data:
                    call_data[data_tipo].append(td_values)
                else:
                    call_data[data_tipo] = [td_values]

    return put_data, call_data

put_data, call_data = extract_data_from_rows(main_soup)
print("Put data:", put_data)
print("Call data:", call_data)


Put data: {'OPE20230421': [['8.700,00', '-', '-', '-', '3,00', '1', '1', '-', '0', '-', '-', '-', '-'], ['8.800,00', '-', '-', '-', '7,00', '2', '1', '-', '0', '-', '-', '-', '-'], ['8.900,00', '-', '-', '-', '3,00', '2', '1', '-', '0', '-', '-', '-', '-'], ['9.000,00', '1', '2', '1,00', '4,00', '1', '1', '-', '0', '-', '-', '-', '-'], ['9.100,00', '2', '2', '1,00', '12,00', '2', '1', '-', '0', '-', '-', '-', '1,00'], ['9.200,00', '1', '2', '1,00', '4,00', '6', '1', '4,00', '1', '4,00', '4,00', '4,00', '3,00'], ['9.300,00', '1', '481', '2,00', '7,00', '501', '2', '7,00', '18', '11,00', '11,00', '7,00', '12,00'], ['9.400,00', '1', '1', '15,00', '19,00', '500', '1', '22,00', '21', '23,00', '30,00', '21,00', '39,00'], ['9.500,00', '1', '1', '53,00', '64,00', '372', '1', '77,00', '4', '88,00', '89,00', '77,00', '96,00'], ['9.600,00', '1', '320', '119,00', '149,00', '320', '1', '140,00', '11', '179,00', '179,00', '135,00', '181,00']], 'OPE20230428': [['7.800,00', '-', '-', '-', '5,00', '300

In [13]:
def extract_data_from_rows(soup):
    rows = soup.find_all('tr', {'class': 'text-right'})
    put_data = {}
    call_data = {}

    for row in rows:
        if 'data-tipo' in row.attrs:
            data_tipo = row['data-tipo']
            td_values = [td.text.strip() for td in row.find_all('td')]
            first_and_last_values = [td_values[0], td_values[-1]]

            if data_tipo.startswith('OPE'):
                if data_tipo in put_data:
                    put_data[data_tipo].append(first_and_last_values)
                else:
                    put_data[data_tipo] = [first_and_last_values]
            elif data_tipo.startswith('OCE'):
                if data_tipo in call_data:
                    call_data[data_tipo].append(first_and_last_values)
                else:
                    call_data[data_tipo] = [first_and_last_values]

    return put_data, call_data

put_data, call_data = extract_data_from_rows(main_soup)
print("Put data:", put_data)
print("Call data:", call_data)


Put data: {'OPE20230421': [['8.700,00', '-'], ['8.800,00', '-'], ['8.900,00', '-'], ['9.000,00', '-'], ['9.100,00', '1,00'], ['9.200,00', '3,00'], ['9.300,00', '12,00'], ['9.400,00', '39,00'], ['9.500,00', '96,00'], ['9.600,00', '181,00']], 'OPE20230428': [['7.800,00', '-'], ['7.900,00', '-'], ['8.000,00', '-'], ['8.100,00', '-'], ['8.200,00', '-'], ['8.300,00', '-'], ['8.400,00', '-'], ['8.500,00', '-'], ['8.600,00', '-'], ['8.700,00', '1,00'], ['8.800,00', '2,00'], ['8.900,00', '3,00'], ['9.000,00', '6,00'], ['9.100,00', '12,00'], ['9.200,00', '23,00'], ['9.300,00', '44,00'], ['9.400,00', '80,00'], ['9.500,00', '141,00'], ['9.600,00', '221,00'], ['9.700,00', '314,00'], ['9.800,00', '412,00'], ['9.900,00', '512,00'], ['10.000,00', '612,00'], ['10.100,00', '711,00'], ['10.200,00', '811,00'], ['10.300,00', '911,00'], ['10.400,00', '1.011,00'], ['10.500,00', '1.111,00'], ['10.600,00', '1.211,00']], 'OPE20230505': [['7.900,00', '-'], ['8.000,00', '1,00'], ['8.100,00', '1,00'], ['8.200,00'

In [14]:
call_data.keys()

dict_keys(['OCE20230421', 'OCE20230428', 'OCE20230505', 'OCE20230512', 'OCE20230519', 'OCE20230616', 'OCE20230915', 'OCE20231215', 'OCE20240315', 'OCE20240621', 'OCE20240920', 'OCE20241220', 'OCE20250321', 'OCE20250620', 'OCE20250919', 'OCE20251219'])

In [15]:
call_data["OCE20230421"]

[['7.500,00', '1.921,00'],
 ['8.400,00', '1.022,00'],
 ['8.500,00', '922,00'],
 ['8.600,00', '822,00'],
 ['8.700,00', '722,00'],
 ['9.000,00', '422,00'],
 ['9.100,00', '323,00'],
 ['9.200,00', '225,00'],
 ['9.300,00', '134,00'],
 ['9.400,00', '61,00'],
 ['9.500,00', '18,00'],
 ['9.600,00', '3,00'],
 ['9.700,00', '-'],
 ['9.800,00', '-'],
 ['9.900,00', '-']]